In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

In [3]:
import src.data as data
import src.utility as utility

In [4]:
e = data.events('test')

caching resources/dataset/preprocessed/events_test.csv.gz


In [5]:
speeds_test = data.speeds_original('test')
speeds_test.shape

caching resources/dataset/originals/speeds_test.csv.gz


(3488814, 9)

In [6]:
joined_df = utility.merge_speed_events(speeds_test, e)

In [7]:
speeds_target = utility.time_windows_event(joined_df, steps_behind=0, steps_after=3)

Total events found: 7319



Filter size: 29276


In [9]:
speeds_target.head()

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration,event_index,sample_id
0,0.0,2018-12-21 17:30:00,337.0,200.463977,31.465900,59.866,276.678,260.0,0_337,2018-12-21 16:00:00,2018-12-21 17:30:00,6.0,Gestione_viabilita,337.0,336.0,NaN,7.0,752.0,0
27383,NaN,2018-12-21 17:45:00,NaN,NaN,NaN,NaN,NaN,NaN,0_337,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,752.0,0
27386,NaN,2018-12-21 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0_337,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,752.0,0
27388,NaN,2018-12-21 18:15:00,NaN,NaN,NaN,NaN,NaN,NaN,0_337,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,752.0,0
1394,0.0,2018-12-07 12:15:00,337.0,218.191853,35.729569,59.866,347.870,231.0,0_337,2018-12-07 11:00:00,2018-12-07 13:00:00,4.0,Ostacolo_in_carreggiata,339.0,332.0,337.0,9.0,174949.0,1


In [10]:
speeds_target.shape

(29276, 19)

In [11]:
speeds_target.dropna(subset=['KEY'], inplace=True)
speeds_target.shape

(27383, 19)

In [24]:
speeds_target['dummy'] = 1
filter_target = speeds_target[['KEY','DATETIME_UTC','KM','dummy']].drop_duplicates()
filter_target.head()

,KEY,DATETIME_UTC,KM,dummy
0,0.0,2018-12-21 17:30:00,337.0,1
1394,0.0,2018-12-07 12:15:00,337.0,1
1398,0.0,2018-12-07 12:30:00,337.0,1
24702,0.0,2018-12-07 12:45:00,337.0,1
21977,0.0,2018-12-07 13:00:00,337.0,1


In [25]:
filter_target.shape

(6882, 4)

In [27]:
filtered = speeds_test.merge(filter_target, how='left')

In [28]:
filtered.shape

(3488814, 10)

In [30]:
filtered[filtered.dummy.isnull()]

,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,dummy
0,0,2018-12-01 05:45:00,443,149.462750,11.339183,137.530,186.070,24,0_443,NaN
1,0,2018-12-01 14:30:00,443,183.164204,18.459800,140.766,223.284,49,0_443,NaN
2,0,2018-12-02 01:30:00,443,172.317000,14.873284,161.800,182.834,2,0_443,NaN
3,0,2018-12-02 08:00:00,443,181.913414,21.211454,139.148,224.902,58,0_443,NaN
4,0,2018-12-03 06:00:00,443,155.443571,19.456223,121.350,218.430,84,0_443,NaN
5,0,2018-12-03 07:00:00,443,169.471552,18.937749,124.586,211.958,116,0_443,NaN
6,0,2018-12-03 10:30:00,443,158.421616,20.744966,69.574,221.666,125,0_443,NaN
7,0,2018-12-03 11:30:00,443,171.076533,18.364618,118.114,218.430,90,0_443,NaN
8,0,2018-12-03 12:30:00,443,161.207267,23.720920,85.754,213.576,101,0_443,NaN
9,0,2018-12-03 20:15:00,443,177.391636,17.565820,152.092,215.194,33,0_443,NaN
